## 基於視窗滑動策略的機器閱讀理解(MRC)
### 下載評估需要的nltk

In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 下載cmrc_eval.py
- 程式有修改,請使用wget下載


In [ ]:
!pip install wget

In [ ]:
#import wget
#wget.download('https://raw.githubusercontent.com/roberthsu2003/Transformer/refs/heads/main/for_download/cmrc_eval.py')

### 載人套件

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForQuestionAnswering,TrainingArguments,Trainer, DefaultDataCollator

### 下載資料集

In [5]:
datasets = load_dataset("roberthsu2003/for_MRC_QA", cache_dir='data')
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 26936
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3524
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3493
    })
})

In [6]:
datasets["train"][0]

{'id': '1001-10-1',
 'context': '2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
 'question': '廣州的快速公交運輸系統每多久就會有一輛巴士？',
 'answers': {'answer_start': [84], 'text': ['10秒鐘']}}

### 數據處理

In [7]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

In [8]:
##選擇前10筆資料處理(讓邏輯簡化)
sample_dataset = datasets['train'].select(range(10))
sample_dataset[:]

{'id': ['1001-10-1',
  '1001-10-2',
  '1001-10-3',
  '1001-11-1',
  '1001-11-2',
  '1001-11-3',
  '1001-12-1',
  '1001-12-2',
  '1001-12-3',
  '1001-13-1'],
 'context': ['2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
  '2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2

In [9]:
#前面放Question,後面放context
#增加requrn_overflowing_tokens
#max_length先設為384,比較好明
from pprint import pprint
tokenized_example = tokenizer(
    text = sample_dataset["question"],
    text_pair=sample_dataset['context'],
    return_offsets_mapping=True,
    return_overflowing_tokens=True,
    max_length=384,
    truncation="only_second",
    padding="max_length"
)
#pprint(tokenized_example,compact=True)
print(tokenized_example.keys()) #多了offset_mapping,overflow_to_sample_mapping

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])


In [10]:
#查看overflow_to_sample_mapping內容(切割資訊)
#一條數據因為長度超過,被切割為多條
tokenized_example['overflow_to_sample_mapping'], len(tokenized_example['overflow_to_sample_mapping']) #本來只有10條變為14條

([0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 9], 14)

In [11]:
#解碼前3條input_ids的資料,看切割後的狀態
#這是沒有overflow的資料
pprint(tokenizer.batch_decode(tokenized_example['input_ids'][:3]),compact=True,width=1000) #改變width查看資料,中文被decode會有1格空格,

['[CLS] 廣 州 的 快 速 公 交 運 輸 系 統 每 多 久 就 會 有 一 輛 巴 士 ？ [SEP] 2010 年 引 進 的 廣 州 快 速 公 交 運 輸 系 統 ， 屬 世 界 第 二 大 快 速 公 交 系 統 ， 日 常 載 客 量 可 達 100 萬 人 次 ， 高 峰 時 期 每 小 時 單 向 客 流 高 達 26900 人 次 ， 僅 次 於 波 哥 大 的 快 速 交 通 系 統 ， 平 均 每 10 秒 鐘 就 有 一 輛 巴 士 ， 每 輛 巴 士 單 向 行 駛 350 小 時 。 包 括 橋 樑 在 內 的 站 台 是 世 界 最 長 的 州 快 速 公 交 運 輸 系 統 站 台 ， 長 達 260 米 。 目 前 廣 州 市 區 的 計 程 車 和 公 共 汽 車 主 要 使 用 液 化 石 油 氣 作 燃 料 ， 部 分 公 共 汽 車 更 使 用 油 電 、 氣 電 混 合 動 力 技 術 。 2012 年 底 開 始 投 放 液 化 天 然 氣 燃 料 的 公 共 汽 車 ， 2014 年 6 月 開 始 投 放 液 化 天 然 氣 插 電 式 混 合 動 力 公 共 汽 車 ， 以 取 代 液 化 石 油 氣 公 共 汽 車 。 2007 年 1 月 16 日 ， 廣 州 市 政 府 全 面 禁 止 在 市 區 內 駕 駛 摩 托 車 。 違 反 禁 令 的 機 動 車 將 會 予 以 沒 收 。 廣 州 市 交 通 局 聲 稱 禁 令 的 施 行 ， 使 得 交 通 擁 擠 問 題 和 車 禍 大 幅 減 少 。 廣 州 白 雲 國 際 機 場 位 於 白 雲 區 與 花 都 區 交 界 ， 2004 年 8 月 5 日 正 式 投 入 運 營 ， 屬 中 國 交 通 情 況 第 二 繁 忙 的 機 場 。 該 機 場 [SEP]',
 '[CLS] 廣 州 的 快 速 公 交 運 輸 系 統 每 多 久 就 會 有 一 輛 巴 士 ？ [SEP] 取 代 了 原 先 位 於 市 中 心 的 無 法 滿 足 日 益 增 長 航 空 需 求 的 舊 機 場 。 目 前 機 場 有 三 條 飛 機 跑 道 ， 成 為 國 內 第 三 個 擁 有 三 跑 道 的 民 航 機 場 。 比 鄰 近 的 香 港 國 際 機

In [12]:
#和上面相同,多一個引數名稱stride(大步)
from pprint import pprint
tokenized_example = tokenizer(
    text = sample_dataset["question"],
    text_pair=sample_dataset['context'],
    return_offsets_mapping=True,
    return_overflowing_tokens=True,
    stride = 128, #設定重疊的部份
    max_length=384,
    truncation="only_second",
    padding="max_length"
)
#pprint(tokenized_example,compact=True)
print(tokenized_example.keys()) #多了offset_mapping,overflow_to_sample_mapping

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])


In [13]:
tokenized_example['overflow_to_sample_mapping'], len(tokenized_example['overflow_to_sample_mapping']) #本來只有10條變為16條,分割同時有重疊

([0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 9], 14)

In [14]:
pprint(tokenizer.batch_decode(tokenized_example['input_ids'][:3]),compact=True,width=100)

['[CLS] 廣 州 的 快 速 公 交 運 輸 系 統 每 多 久 就 會 有 一 輛 巴 士 ？ [SEP] 2010 年 引 進 的 廣 州 快 速 公 交 運 輸 系 統 ， 屬 世 界 '
 '第 二 大 快 速 公 交 系 統 ， 日 常 載 客 量 可 達 100 萬 人 次 ， 高 峰 時 期 每 小 時 單 向 客 流 高 達 26900 人 次 ， 僅 次 於 波 哥 大 '
 '的 快 速 交 通 系 統 ， 平 均 每 10 秒 鐘 就 有 一 輛 巴 士 ， 每 輛 巴 士 單 向 行 駛 350 小 時 。 包 括 橋 樑 在 內 的 站 台 是 世 界 最 長 '
 '的 州 快 速 公 交 運 輸 系 統 站 台 ， 長 達 260 米 。 目 前 廣 州 市 區 的 計 程 車 和 公 共 汽 車 主 要 使 用 液 化 石 油 氣 作 燃 料 ， 部 '
 '分 公 共 汽 車 更 使 用 油 電 、 氣 電 混 合 動 力 技 術 。 2012 年 底 開 始 投 放 液 化 天 然 氣 燃 料 的 公 共 汽 車 ， 2014 年 6 月 開 '
 '始 投 放 液 化 天 然 氣 插 電 式 混 合 動 力 公 共 汽 車 ， 以 取 代 液 化 石 油 氣 公 共 汽 車 。 2007 年 1 月 16 日 ， 廣 州 市 政 府 全 '
 '面 禁 止 在 市 區 內 駕 駛 摩 托 車 。 違 反 禁 令 的 機 動 車 將 會 予 以 沒 收 。 廣 州 市 交 通 局 聲 稱 禁 令 的 施 行 ， 使 得 交 通 擁 擠 '
 '問 題 和 車 禍 大 幅 減 少 。 廣 州 白 雲 國 際 機 場 位 於 白 雲 區 與 花 都 區 交 界 ， 2004 年 8 月 5 日 正 式 投 入 運 營 ， 屬 中 國 交 '
 '通 情 況 第 二 繁 忙 的 機 場 。 該 機 場 [SEP]',
 '[CLS] 廣 州 的 快 速 公 交 運 輸 系 統 每 多 久 就 會 有 一 輛 巴 士 ？ [SEP] 氣 公 共 汽 車 。 2007 年 1 月 16 日 ， 廣 州 市 政 府 '
 '全 面 禁 止 在 市 區 內 駕 駛 摩 托 車 。 違 反 禁 令 的 機 動 車 將 會 予 以 沒 收 。

In [15]:
pprint(tokenized_example['offset_mapping'][:3],compact=True,width=500)

[[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (0, 0), (0, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32),
  (32, 33), (33, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 42), (42, 43), (43, 44), (44, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 62), (62, 64), (64, 65), (65, 66), (66, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 86), (86, 87),
  (87, 88), 

In [16]:
#取出overflow_mapping
sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
sample_mapping

[0, 0, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 9]

In [18]:
for idx, _ in enumerate(sample_mapping):    
    answer = sample_dataset['answers'][sample_mapping[idx]] #參考白板比較好理解
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer['text'][0])

    context_start = tokenized_example.sequence_ids(idx).index(1)
    context_end = tokenized_example.sequence_ids(idx).index(None,context_start)-1

    offset = tokenized_example.get("offset_mapping")[idx]

    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        #答案不在context內
        start_token_pos = 0
        end_token_pos = 0
    else:
        #由左而右再由右而左找尋答案的index
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -= 1
        end_token_pos = token_id

    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenized_example['input_ids'][idx][start_token_pos:end_token_pos + 1]))


#觀察有些找不到資料:[CLS]

{'answer_start': [84], 'text': ['10秒鐘']} 84 88 24 382 100 102
token answer decode: 10 秒 鐘
{'answer_start': [84], 'text': ['10秒鐘']} 84 88 24 235 0 0
token answer decode: [CLS]
{'answer_start': [256], 'text': ['2007年1月16日']} 256 266 22 382 259 264
token answer decode: 2007 年 1 月 16 日
{'answer_start': [256], 'text': ['2007年1月16日']} 256 266 22 231 26 31
token answer decode: 2007 年 1 月 16 日
{'answer_start': [447], 'text': ['香港國際機場']} 447 453 35 382 0 0
token answer decode: [CLS]
{'answer_start': [447], 'text': ['香港國際機場']} 447 453 35 257 236 241
token answer decode: 香 港 國 際 機 場
{'answer_start': [104], 'text': ['200公里']} 104 109 19 286 113 115
token answer decode: 200 公 里
{'answer_start': [166], 'text': ['兩小時']} 166 169 18 285 170 172
token answer decode: 兩 小 時
{'answer_start': [212], 'text': ['渡輪線路']} 212 216 27 294 225 228
token answer decode: 渡 輪 線 路
{'answer_start': [49], 'text': ['廣州']} 49 51 26 309 69 70
token answer decode: 廣 州
{'answer_start': [163], 'text': ['廣州']} 163 165 20 303 166

In [96]:
#建立處理token的function
def process_func(examples):
    tokenized_example = tokenizer(
        text = examples["question"],
        text_pair=examples['context'],
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        stride = 128, #設定重疊的部份
        max_length=384,
        truncation="only_second",
        padding="max_length"
        )
    sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []
    example_ids = []
    
    for idx, _ in enumerate(sample_mapping):    
        answer = examples['answers'][sample_mapping[idx]] #參考白板比較好理解
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer['text'][0])

        context_start = tokenized_example.sequence_ids(idx).index(1)
        context_end = tokenized_example.sequence_ids(idx).index(None,context_start)-1

        offset = tokenized_example.get("offset_mapping")[idx]

        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            #答案不在context內
            start_token_pos = 0
            end_token_pos = 0
        else:
            #由左而右再由右而左找尋答案的index
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -= 1
            end_token_pos = token_id

        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
        example_ids.append(examples["id"][sample_mapping[idx]])
        #這些程式碼是為了預測使用的
        tokenized_example["offset_mapping"][idx] = [
            (o if tokenized_example.sequence_ids(idx)[k] == 1 else None)
            for k, o in enumerate(tokenized_example["offset_mapping"][idx])
        ]


    tokenized_example["example_ids"] = example_ids
    tokenized_example["start_positions"] = start_positions
    tokenized_example["end_positions"] = end_positions
    return tokenized_example



#觀察有些找不到資料:[CLS]

In [97]:
datasets = load_dataset("roberthsu2003/for_MRC_QA", cache_dir='data')
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')


In [104]:
tokenized_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/26936 [00:00<?, ? examples/s]

In [110]:
print(tokenized_datasets['train'])
print(tokenized_datasets['train']['offset_mapping'][10])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
    num_rows: 47705
})
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [31, 32], [32, 36], [36, 37], [37, 38], [38, 39], [39, 40], [40, 41], [41, 42], [42, 43], [43, 44], [44, 45], [45, 46], [46, 47], [47, 48], [48, 49], [49, 50], [50, 51], [51, 52], [52, 53], [53, 54], [54, 55], [55, 56], [56, 57], [57, 58], [58, 62], [62, 63], [63, 64], [64, 65], [65, 66], [66, 67], [67, 68], [68, 69], [69, 70], [70, 71], [71, 72], [72, 75], [75, 77], [77, 78], [78, 80], [80, 81], [81, 82], [82, 83], [83, 84], [84, 85], [85, 86], [

In [114]:
import collections
# example 和 feature的映射
example_to_feature = collections.defaultdict(list)
for idx, example_id in enumerate(tokenized_datasets["train"]["example_ids"][:10]):
    example_to_feature[example_id].append(idx)
example_to_feature

defaultdict(list,
            {'1001-10-1': [0, 1],
             '1001-10-2': [2, 3],
             '1001-10-3': [4, 5],
             '1001-11-1': [6],
             '1001-11-2': [7],
             '1001-11-3': [8],
             '1001-12-1': [9]})

### 獲取模型輸出

In [116]:
import numpy as np
import collections

def get_result(start_logits, end_logits, exmaples, features):

    predictions = {}
    references = {}

    # example 和 feature的映射
    example_to_feature = collections.defaultdict(list)
    for idx, example_id in enumerate(features["example_ids"]):
        example_to_feature[example_id].append(idx)

    # 最优答案候选
    n_best = 20
    # 最大答案长度
    max_answer_length = 30

    for example in exmaples:
        example_id = example["id"]
        context = example["context"]
        answers = []
        for feature_idx in example_to_feature[example_id]:
            start_logit = start_logits[feature_idx]
            end_logit = end_logits[feature_idx]
            offset = features[feature_idx]["offset_mapping"]
            start_indexes = np.argsort(start_logit)[::-1][:n_best].tolist()
            end_indexes = np.argsort(end_logit)[::-1][:n_best].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offset[start_index] is None or offset[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    answers.append({
                        "text": context[offset[start_index][0]: offset[end_index][1]],
                        "score": start_logit[start_index] + end_logit[end_index]
                    })
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else:
            predictions[example_id] = ""
        references[example_id] = example["answers"]["text"]

    return predictions, references

### 評估函數

In [117]:
from cmrc_eval import evaluate_cmrc

def metirc(pred):
    start_logits, end_logits = pred[0]
    if start_logits.shape[0] == len(tokenized_datasets["validation"]):
        p, r = get_result(start_logits, end_logits, datasets["validation"], tokenized_datasets["validation"])
    else:
        p, r = get_result(start_logits, end_logits, datasets["test"], tokenized_datasets["test"])
    return evaluate_cmrc(p, r)

### 下載模型

In [118]:
model = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-base-chinese")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 配置TrainingArguments

In [119]:
args = TrainingArguments(
    output_dir="models_for_qa_slide",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=1
)

### Step8 配置Trainer

In [123]:
trainer = Trainer(
    model=model,
    args=args,
    processing_class=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DefaultDataCollator(),
    compute_metrics=metirc
)

### 模型訓練

In [124]:
trainer.train()

: 

### 模型预测

In [ ]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
pipe

In [ ]:
pipe(question="小明在哪里上班？", context="小明在北京上班")